In [ ]:
from utils.random_board import generate_random_board
from utils.stockfish import gest_best_move
import os

board = generate_random_board()
print(board)
print(board.legal_moves)
move = gest_best_move(board, 10)
print(move)


In [ ]:
import tensorflow as tf

tensor_test = tf.random.uniform((10000,14,8,8), minval=0, maxval=2, dtype=tf.int32)
tensor_test_2 = tf.random.uniform((10000,14,8,8), minval=0, maxval=2, dtype=tf.int32)

In [ ]:
import chess
import numpy as np

def convert_square_to_coordinates(square: chess.Square):
    # ex h1 -> (7, 0)
    print(square)
    return (7 - square // 8, square % 8)

print(convert_square_to_coordinates(chess.H1))


In [1]:
from utils.process_board import convert_board_to_array
from utils.random_board import generate_random_board
from utils.stockfish import get_score
from utils.process_tensors import save_tensor, load_tensor, concat_tensors
import numpy as np
import tensorflow as tf

def generate_data(n=100, N=25, folder_name=''):

    save_dir = f"data/{folder_name}/"
    input_path = save_dir + "input"
    output_path = save_dir + "output"

    for k in range(n):
        input_array = np.zeros((N, 7, 8, 8), dtype=np.int32)
        output_array = np.zeros((N,), dtype=np.float32)
        for i in range(N):
            board = generate_random_board()
            input_array[i] = convert_board_to_array(board)
            output_array[i] = get_score(board, 10)

        input_tensor = tf.convert_to_tensor(input_array)
        output_tensor = tf.convert_to_tensor(output_array)
        try:
            loaded_input = load_tensor(input_path, out_type=tf.int32)
            loaded_output = load_tensor(output_path, out_type=tf.float32)

            input_tensor = concat_tensors([input_tensor, loaded_input], axis=0)
            output_tensor = concat_tensors([output_tensor, loaded_output], axis=0)

        except Exception as e:
            print('erreur', e)
            pass
        save_tensor(input_tensor, input_path)
        save_tensor(output_tensor, output_path)
        
        print(folder_name, (k+1)*N)

In [2]:
import threading
threading.Thread(target=generate_data, args=(400,25, 'test1')).start()
threading.Thread(target=generate_data, args=(400,25, 'test2')).start()
threading.Thread(target=generate_data, args=(400,25, 'test3')).start()
threading.Thread(target=generate_data, args=(400,25, 'test4')).start()

test2 25
test4 25
test3 25
test 25
test2 50
test4 50
test3 50
test 50
test2 75
test4 75
test3 75
test 75
test2 100
test4 100
test3 100
test 100
test2 125
test4 125
test3 125
test 125
test2 150
test4 150
test3 150
test 150
test4 175
test2 175
test3 175
test 175
test4 200
test2 200
test3 200
test 200
test4 225
test2 225
test3 225
test 225
test4 250
test2 250
test3 250
test 250
test4 275
test2 275
test3 275
test 275
test4 300
test2 300
test3 300
test 300
test4 325
test2 325
test3 325
test 325
test4 350
test2 350
test3 350
test 350
test4 375
test2 375
test3 375
test 375
test4 400
test2 400
test3 400
test 400
test4 425
test2 425
test3 425
test 425
test4 450
test2 450
test3 450
test 450
test4 475
test2 475
test3 475
test 475
test4 500
test2 500
test3 500
test 500
test4 525
test2 525
test3 525
test 525
test4 550
test2 550
test3 550
test 550
test4 575
test2 575
test3 575
test 575
test4 600
test3 600
test2 600
test 600
test4 625
test2 625
test3 625
test4 650
test3 650
test2 650
test4 675
test 6

In [16]:
load_dir = "data/test/"
input_path = load_dir + "input"
output_path = load_dir + "output"

input_tensor = load_tensor(input_path, out_type=tf.int32)
output_tensor = load_tensor(output_path, out_type=tf.float32)

In [17]:
output_tensor
normalized_tensor = tf.linalg.normalize(output_tensor, axis=0, ord='euclidean', name=None)[0]
print(normalized_tensor.shape)


(19790,)


In [ ]:
# print max value of tensor

print(normalized_tensor.shape)
maxi = tf.math.reduce_max(normalized_tensor)
mini = tf.math.reduce_min(normalized_tensor)
mean = tf.math.reduce_mean(normalized_tensor)
std = tf.math.reduce_std(normalized_tensor)
 
print('max', maxi)
print('min', mini)
print('mean', mean)
print('std', std)

test = normalized_tensor/maxi

print(tf.math.reduce_max(test))
print(tf.math.reduce_min(test))


In [ ]:
max = tf.math.reduce_max(output_tensor)
min = tf.math.reduce_min(output_tensor)
std = tf.math.reduce_std(output_tensor)

abs_max = tf.math.maximum(tf.math.abs(max), tf.math.abs(min))
print(abs_max)
print(output_tensor/abs_max)
